
# RNN using LSTM 
       




These lessons are adapted from [leriomaggio deep learning keras TensorFlow tutorials](https://github.com/leriomaggio/deep-learning-keras-tensorflow) 
 / [GitHub](https://github.com/leriomaggio/deep-learning-keras-tensorflow)  
which are published under the [MIT License](https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/LICENSE) which allows very broad use for both academic and commercial purposes.

<img src="../imgs/RNN-rolled.png"/ width="80px" height="80px">

<img src="../imgs/RNN-unrolled.png"/ width="400px" height="400px">

<img src="../imgs/LSTM3-chain.png"/ width="60%">

_source: http://colah.github.io/posts/2015-08-Understanding-LSTMs_

In [1]:
from keras.optimizers import SGD
from keras.preprocessing.text import one_hot, text_to_word_sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.preprocessing import sequence

Using TensorFlow backend.


### Reading blog post from data directory

In [2]:
import os
import pickle
import numpy as np

In [3]:
DATA_DIRECTORY = os.path.join(os.path.abspath(os.path.curdir), '..', 'data', 'word_embeddings')
print(DATA_DIRECTORY)

/Users/bear/Documents/NEU_COE/Deep_Learning/RNNs/../data/word_embeddings


In [4]:
male_posts = []
female_post = []

In [5]:
with open(os.path.join(DATA_DIRECTORY,"male_blog_list.txt"),"rb") as male_file:
    male_posts= pickle.load(male_file)
    
with open(os.path.join(DATA_DIRECTORY,"female_blog_list.txt"),"rb") as female_file:
    female_posts = pickle.load(female_file)

In [6]:
filtered_male_posts = list(filter(lambda p: len(p) > 0, male_posts))
filtered_female_posts = list(filter(lambda p: len(p) > 0, female_posts))

In [7]:
# text processing - one hot builds index of the words
male_one_hot = []
female_one_hot = []
n = 30000
for post in filtered_male_posts:
    try:
        male_one_hot.append(one_hot(post, n, split=" ", lower=True))
    except:
        continue

for post in filtered_female_posts:
    try:
        female_one_hot.append(one_hot(post,n,split=" ", lower=True))
    except:
        continue

In [8]:
# 0 for male, 1 for female
concatenate_array_rnn = np.concatenate((np.zeros(len(male_one_hot)),
                                        np.ones(len(female_one_hot))))

In [9]:
from sklearn.model_selection import train_test_split

X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(np.concatenate((female_one_hot,male_one_hot)),
                                                                    concatenate_array_rnn, 
                                                                    test_size=0.2)

In [10]:
maxlen = 100
X_train_rnn = sequence.pad_sequences(X_train_rnn, maxlen=maxlen)
X_test_rnn = sequence.pad_sequences(X_test_rnn, maxlen=maxlen)
print('X_train_rnn shape:', X_train_rnn.shape, y_train_rnn.shape)
print('X_test_rnn shape:', X_test_rnn.shape, y_test_rnn.shape)

X_train_rnn shape: (3873, 100) (3873,)
X_test_rnn shape: (969, 100) (969,)


In [11]:
max_features = 30000
dimension = 128
output_dimension = 128
model = Sequential()
model.add(Embedding(max_features, dimension))
model.add(LSTM(output_dimension))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [12]:
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])

In [13]:
model.fit(X_train_rnn, y_train_rnn, batch_size=32,
          epochs=4, validation_data=(X_test_rnn, y_test_rnn))

Train on 3873 samples, validate on 969 samples
Epoch 1/4
3873/3873 [==============================] - 29s - loss: 0.2499 - acc: 0.5161 - val_loss: 0.2492 - val_acc: 0.5542
Epoch 2/4
3873/3873 [==============================] - 30s - loss: 0.2494 - acc: 0.5309 - val_loss: 0.2486 - val_acc: 0.5521
Epoch 3/4
3873/3873 [==============================] - 30s - loss: 0.2493 - acc: 0.5319 - val_loss: 0.2484 - val_acc: 0.5521
Epoch 4/4
3873/3873 [==============================] - 31s - loss: 0.2492 - acc: 0.5319 - val_loss: 0.2480 - val_acc: 0.5521


In [14]:
score, acc = model.evaluate(X_test_rnn, y_test_rnn, batch_size=32)

960/969 [============================>.] - ETA: 0s

In [15]:
print(score, acc)

0.24798820979 0.55211558326


# Using TFIDF Vectorizer as an input instead of one hot encoder

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
vectorizer = TfidfVectorizer(decode_error='ignore', norm='l2', min_df=5)
tfidf_male = vectorizer.fit_transform(filtered_male_posts)
tfidf_female = vectorizer.fit_transform(filtered_female_posts)

In [18]:
flattened_array_tfidf_male = tfidf_male.toarray()
flattened_array_tfidf_female = tfidf_male.toarray()

In [19]:
y_rnn = np.concatenate((np.zeros(len(flattened_array_tfidf_male)),
                        np.ones(len(flattened_array_tfidf_female))))

In [20]:
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(np.concatenate((flattened_array_tfidf_male, 
                                                                                    flattened_array_tfidf_female)),
                                                                    y_rnn,test_size=0.2)

In [21]:
maxlen = 100
X_train_rnn = sequence.pad_sequences(X_train_rnn, maxlen=maxlen)
X_test_rnn = sequence.pad_sequences(X_test_rnn, maxlen=maxlen)
print('X_train_rnn shape:', X_train_rnn.shape, y_train_rnn.shape)
print('X_test_rnn shape:', X_test_rnn.shape, y_test_rnn.shape)

X_train_rnn shape: (4152, 100) (4152,)
X_test_rnn shape: (1038, 100) (1038,)


In [22]:
max_features = 30000
model = Sequential()
model.add(Embedding(max_features, dimension))
model.add(LSTM(output_dimension))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [23]:
model.compile(loss='mean_squared_error',optimizer='sgd', metrics=['accuracy'])

In [24]:
model.fit(X_train_rnn, y_train_rnn, 
          batch_size=32, epochs=1,
          validation_data=(X_test_rnn, y_test_rnn))

Train on 4152 samples, validate on 1038 samples
Epoch 1/1
4152/4152 [==============================] - 34s - loss: 0.2501 - acc: 0.4880 - val_loss: 0.2500 - val_acc: 0.4798


In [25]:
score,acc = model.evaluate(X_test_rnn, y_test_rnn, 
                           batch_size=32)

1038/1038 [==============================] - 1s     


In [26]:
print(score, acc)

0.250002229834 0.479768786127


# Sentence Generation using LSTM

In [27]:
# reading all the male text data into one string
male_post = ' '.join(filtered_male_posts)

#building character set for the male posts
character_set_male = set(male_post)
#building two indices - character index and index of character
char_indices = dict((c, i) for i, c in enumerate(character_set_male))
indices_char = dict((i, c) for i, c in enumerate(character_set_male))


# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 1
sentences = []
next_chars = []
for i in range(0, len(male_post) - maxlen, step):
    sentences.append(male_post[i : i + maxlen])
    next_chars.append(male_post[i + maxlen])


In [28]:
#Vectorisation of input
x_male = np.zeros((len(male_post), maxlen, len(character_set_male)), dtype=np.bool)
y_male = np.zeros((len(male_post), len(character_set_male)), dtype=np.bool)

print(x_male.shape, y_male.shape)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x_male[i, t, char_indices[char]] = 1
    y_male[i, char_indices[next_chars[i]]] = 1

print(x_male.shape, y_male.shape)

(2552476, 20, 152) (2552476, 152)
(2552476, 20, 152) (2552476, 152)


In [30]:

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(character_set_male))))
model.add(Dense(len(character_set_male)))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Build model...


In [32]:
import random, sys

In [ ]:
# helper function to sample an index from a probability array
def sample(a, diversity=0.75):
    if random.random() > diversity:
        return np.argmax(a)
    while 1:
        i = random.randint(0, len(a)-1)
        if a[i] > random.random():
            return i

In [ ]:
# train the model, output generated text after each iteration
for iteration in range(1,10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_male, y_male, batch_size=128, epochs=1)

    start_index = random.randint(0, len(male_post) - maxlen - 1)

    for diversity in [0.2, 0.4, 0.6, 0.8]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = male_post[start_index : start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')

        for iteration in range(400):
            try:
                x = np.zeros((1, maxlen, len(character_set_male)))
                for t, char in enumerate(sentence):
                    x[0, t, char_indices[char]] = 1.

                preds = model.predict(x, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char
            except:
                continue
                
        print(sentence)
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
2552476/2552476 [==============================] - 1958s - loss: 2.1010 - acc: 0.4116  

----- diversity: 0.2
----- Generating with seed: "y team was quite rea"
stull the sears and 


----- diversity: 0.4
----- Generating with seed: "y team was quite rea"
ory and the and the 


----- diversity: 0.6
----- Generating with seed: "y team was quite rea"
e up them to feating


----- diversity: 0.8
----- Generating with seed: "y team was quite rea"
  The blat way to ex


--------------------------------------------------
Iteration 2
Epoch 1/1
2552476/2552476 [==============================] - 2720s - loss: 1.7487 - acc: 0.4985  

----- diversity: 0.2
----- Generating with seed: "te! i need a counter"
have to see the take


----- diversity: 0.4
----- Generating with seed: "te! i need a counter"
op the star and the 


----- diversity: 0.6
----- Generating with seed: "te! i need a counter"
h that all, the the 


----- dive